In [8]:
!pip install langchain openai langchain_openai langchain_core langchain_chains

ERROR: Could not find a version that satisfies the requirement langchain_chains (from versions: none)
ERROR: No matching distribution found for langchain_chains


In [9]:
import os
import openai

os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

In [24]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4")

In [25]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi! how are you doing?"),
]

model.invoke(messages)

AIMessage(content='Ciao! Come stai?', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 25, 'total_tokens': 32}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c843ccd6-1fe3-45e6-88f8-55b8487fde0c-0', usage_metadata={'input_tokens': 25, 'output_tokens': 7, 'total_tokens': 32})

**Parsing the output**

In [27]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [28]:
result = model.invoke(messages)
parser.invoke(result)

'Ciao! Come stai?'

**Prompt Template**

In [29]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi, how are you?"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi, how are you?')])

In [30]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi, how are you?')]

**Chaining together components with LCEL**

In [32]:
message = 'hi, how are you?'

chain = prompt_template | model | parser
chain.invoke({"language": "french", "text": message})

'salut, comment ça va?'

### **Building a ChatBot**

In [33]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [34]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f7a92158-2210-44cd-ac42-11422c0f4447-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

In [35]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not have access to personal information such as your name. Can I assist you with anything else?", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 12, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-39b9ec3f-0e44-4554-81a6-0669094eaa31-0', usage_metadata={'input_tokens': 12, 'output_tokens': 25, 'total_tokens': 37})

In [36]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f154b7b8-ef88-499b-8df4-d0b96936b078-0', usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40})

**Message History**
<BR>
We can use a Message History class to wrap our model and make it stateful

In [37]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [38]:
config = {"configurable": {"session_id": "abc2"}}

In [41]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

"Hello Bob! It seems like you keep introducing yourself as Bob. Is there a reason for that or is there something you'd like to discuss?"

In [42]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob, based on the repeated introduction you provided. How can I assist you today, Bob?'

Now the chatbot  **remembers** things about us. <br>
If we change the config to reference a different session_id, we can see that it starts the conversation fresh.

In [43]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I don't know your name. Can you please tell me?"

In [44]:
store

{'abc2': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Bob"), AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cb341436-bee4-451e-a954-525e46bf9f0f-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22}), HumanMessage(content="Hi! I'm Bob"), AIMessage(content='Hello Bob! It seems like you introduced yourself as Bob again. Is there anything specific you would like to talk about or ask today?', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 35, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-15e76be7-9071-42f9-93b5-a5913d26f018-0', usage_metadata={'input_tokens': 35, 'output_tokens': 27, 'total_token